In [19]:
!pip install -q kaggle


In [20]:
! mkdir -p ~/.kaggle
! cp ~/kaggle.json ~/.kaggle/   # adjust path if needed
! chmod 600 ~/.kaggle/kaggle.json

In [21]:
# Where is HOME for this kernel?
!echo $HOME

# Does the file exist and have 600 perms?
!ls -l ~/.kaggle
!ls -l ~/.kaggle/kaggle.json

# Peek at the *shape* (confirm keys exist). DO NOT share output publicly.
!cat ~/.kaggle/kaggle.json

/Users/alex
total 8
-rw-------@ 1 alex  staff  73 Aug 25 15:46 kaggle.json
-rw-------@ 1 alex  staff  73 Aug 25 15:46 /Users/alex/.kaggle/kaggle.json
{"username":"alextheambitious","key":"cbe0ae38313eef2ff35a847ae89ec723"}


In [23]:
!kaggle competitions download -c mercari-price-suggestion-challenge -p data/raw --force

 99%|███████████████████████████████████████▌| 399M/403M [00:08<00:00, 52.9MB/s]
100%|████████████████████████████████████████| 403M/403M [00:08<00:00, 51.9MB/s]


In [30]:
!unzip "data/raw/mercari-price-suggestion-challenge.zip" -d "data/raw"

Archive:  data/raw/mercari-price-suggestion-challenge.zip
  inflating: data/raw/sample_submission.csv.7z  
  inflating: data/raw/sample_submission_stg2.csv.zip  
  inflating: data/raw/test.tsv.7z    
  inflating: data/raw/test_stg2.tsv.zip  
  inflating: data/raw/train.tsv.7z   


Running `brew update --auto-update`...
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:20fa657858e44a4b39171d6e4111f8a9716eb62a78ebbd1491d94f90bb7b830a
######################################################################### 100.0%
==> Pouring portable-ruby-3.4.5.arm64_big_sur.bottle.tar.gz
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorded yet (nor will be during this `brew` run).

==> Homebrew is run entirely by unpaid volunteers. Please consider donating:
  https://github.com/Homebrew/brew#donations

==> homebrew/cask is old and unneeded, untapping to save space...
Untapping homebrew/cask...
Untapped 7579 casks (4,321 files, 365.5MB).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
ab-av1: AV1 re-encoding using ffmpeg, svt-av1 & vmaf
abpoa: SIMD-based C library for fast partial order 

In [37]:
!7z x data/raw/train.tsv.7z -o./data/raw/dataset
!7z x data/raw/test.tsv.7z -o./data/raw/dataset


7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,10 CPUs LE)

Scanning the drive for archives:
1 file, 77912192 bytes (75 MiB)

Extracting archive: data/raw/train.tsv.7z
--
Path = data/raw/train.tsv.7z
Type = 7z
Physical Size = 77912192
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

Everything is Ok

Size:       337809843
Compressed: 77912192

7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,10 CPUs LE)

Scanning the drive for archives:
1 file, 35617013 bytes (34 MiB)

Extracting archive: data/raw/test.tsv.7z
--
Path = data/raw/test.tsv.7z
Type = 7z
Physical Size = 35617013
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

Everything is Ok

Size:       154222160
Compressed: 35617013


In [1]:
import pandas as pd
from src.feature.feature_engineering import FeatureEngineer, FeatureEngineerConfig

# 1. Load dataset
df = pd.read_csv("data/raw/dataset/train.tsv", sep="\t")

# 2. Create FeatureEngineer with your thresholds
cfg = FeatureEngineerConfig(
    min_brand_low=10,
    min_brand_mid=50,
    min_cat3=50,
    add_brand_bucket=True,
    replace_rare_brands=False,
    make_log_price=True
)

# 3. Fit on the whole train.tsv
fe = FeatureEngineer(cfg).fit(df)

# 4. Transform the same df (just to inspect results)
df_fe = fe.transform(df)

print(df_fe.head())
print(df_fe.columns)

/Users/alex/Desktop/project1/reselling-tools/Clothes AI/src/feature/feature_engineering.py:167: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["year_present"] = (out["name"].str.contains(year_re) |
/Users/alex/Desktop/project1/reselling-tools/Clothes AI/src/feature/feature_engineering.py:168: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["item_description"].str.contains(year_re)).astype("int8")


   train_id                                 name  item_condition_id  \
0         0  mlb cincinnati reds t shirt size xl                  3   
1         1     razer blackwidow chroma keyboard                  3   
2         2                       ava-viv blouse                  1   
3         3                leather horse statues                  1   
4         4                 24k gold plated rose                  1   

                                       category_name brand_name  price  \
0                                  men/tops/t-shirts    unknown   10.0   
1  electronics/computers & tablets/components & p...      razer   52.0   
2                        women/tops & blouses/blouse     target   10.0   
3                 home/home décor/home décor accents    unknown   35.0   
4                            women/jewelry/necklaces    unknown   44.0   

   shipping                                   item_description         cat1  \
0         1                                 no de

In [2]:
from sklearn.model_selection import train_test_split

# Step 1: Split train vs temp (85% train, 15% temp)
train_df, temp_df = train_test_split(
    df_fe,
    test_size=0.30,            # 30% goes to temp (val+test)
    stratify=df_fe["cat1"],       # balance across top categories
    random_state=42
)

# Step 2: Split temp into validation and test (50/50 split)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,             # half temp goes to test, half to val
    stratify=temp_df["cat1"],
    random_state=42
)
train_df = train_df[train_df["cat1"] != "electronics"].copy()
val_df   = val_df[val_df["cat1"] != "electronics"].copy()
test_df  = test_df[test_df["cat1"] != "electronics"].copy()

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

Train shape: (951891, 23)
Validation shape: (203977, 23)
Test shape: (203977, 23)


In [3]:
train_df.to_csv("data/processed/train.csv", index=False)
val_df.to_csv("data/processed/val.csv", index=False)
test_df.to_csv("data/processed/test.csv", index=False)